# Data Fund Data Analysis

We will be looking at the data released by the State Bank of Pakistan. The data is the daily bank-wise and donor-wise receipts of the fund for the Daimer Bhasha and Mohmand Dam. You can find them in the following link: http://www.sbp.org.pk/notifications/FD/DamFund/Damfund.htm

We will see how to scrape data from the web, clean it, and import it into Pandas for data analysis purposes. There are however some things to note:
1. As you can tell, the data is in PDF form. PDF is the most difficult to handle data format and if you get extremely broken CSV files, there isn't a need to worry, that's where the cleaning part comes in.
2. We'll be using an API to convert the data from PDF to CSV, and then from CSV to Pandas. There are, however, other ways to do this. The reason we wanted to do this method is two-fold
3. We will use regular-expressions to clean the data since the CSV's will be extremely inconsistent.

## Part 0: Getting the Data

We will download the data from the link http://www.sbp.org.pk/notifications/FD/DamFund/Damfund.htm

Questions to think about while going through the data:

1. Are there any inconsistencies? 
2. Any particular values that pop out that would need to be taken care of later in the cleaning process?
3. How many columns are there, each, in the PDF files? 

## Part 1: Data Scraping              

We'll be using what is called the *requests* model to get an HTML page, and then use *BeautifulSoup* to parse that HTML page such that we are able to to derive the appropriate information from it. 

* [Requests Documentation](http://docs.python-requests.org/en/master/)
* [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [BeautifulSoup + Requests Tutorial](https://www.pythonforbeginners.com/python-on-the-web/web-scraping-with-beautifulsoup)
* [BeautifulSoup](https://medium.freecodecamp.org/how-to-scrape-websites-with-python-and-beautifulsoup-5946935d93fe) 

Above are the links for the web scraping libraries documentation.

In [1]:
# Import libraries
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
import os
from urllib.request import urlretrieve
from urllib.parse import urljoin
from tqdm import tqdm_notebook

import config
import glob
from pprint import pprint

import csv
import re # To use regular expressions
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime

In [2]:
directory = 'SBF/'
# os is being imported so you can make a new directory. 
if not os.path.exists('./' + directory):
    os.mkdir(directory)

In [2]:
# Open each PDF link
# Save the PDF in a directory in the same folder 
    
url = 'http://www.sbp.org.pk/notifications/FD/DamFund/Damfund.htm'
r  = requests.get(url)
# print(r.status_code)

data = r.content
soup = BeautifulSoup(data, 'html.parser')
# print(soup.prettify())

for links in tqdm_notebook(soup.find_all('a')):
    href = links.get('href')
    if href.endswith('pdf'):
        pattern = re.findall(r'\d{2}-\d{2}-\d{4}\.\w+', href)[0]
        if not os.path.exists(directory + pattern):
            # Skip downloading if file is already present
            pdf_link = urljoin(url, href)
            pdf_response = requests.get(pdf_link)
            with open(directory + pattern, 'wb') as f:
                f.write(pdf_response.content)
        
print(f'{len(os.listdir(directory))} pdf files downloaded')


59 pdf files downloaded


## Part 2: Converting from PDF to CSV

We are using the Zamzar API with 2 accounts. We start jobs for all PDFs untill the credits are exhausted. We then download the CSVs when they're done. Now we cange the API key to point to the other acount and only post jobs for files which have not been downloaded yet. Due to the number of files we have in out dataset just 2 turns with teh Zamzar API are sufficient.  
  
*There was one file `27-08-2018.pdf` larger than 1MB which is the file size limit for free accounts on Zamzar. We used an online PDF compressor at `https://www.ilovepdf.com/compress_pdf` to bring the file size down to around 300KB. It was verified that the compressed file had no differences in text content.*

In [4]:
# Zamzar API authentication
api_key = '9e485cc256a330127609b10172392af7139683df'
auth_token = HTTPBasicAuth(api_key, '')

endpoint = "https://api.zamzar.com/v1/account"
res = requests.get(endpoint, auth=auth_token)
pprint(res.json())

{'credits_remaining': 28,
 'plan': {'conversions_per_month': 100,
          'maximum_file_size': 1048576,
          'name': 'Test',
          'price_per_month': 0},
 'test_credits_remaining': 100}


In [3]:
csv_directory = directory + 'csv/'
already_done = set(f.partition('.')[0] for f in os.listdir(csv_directory)) \
               if os.path.exists(csv_directory) else set()
print(f'{len(already_done)} are already downloaded.')

58 are already downloaded.


In [5]:
# You need a list to store all the job_ids from the response of posting the conversion job, 
# if you are using the Zamzar API
job_ids = []

for file_name in tqdm_notebook(glob.glob(directory + '*.pdf')):
    ## Write code here to post a job, and append each job's id into job_ids ##
    endpoint = "https://api.zamzar.com/v1/jobs"
    with open(file_name, 'rb') as f:
        just_name = file_name.partition('\\')[2].partition('.')[0]
        if just_name in already_done:
            continue
        
        file_content = {'source_file': f}
        data_content = {'target_format': 'csv'}
        res = requests.post(endpoint, 
                            data=data_content, 
                            files=file_content, 
                            auth=auth_token)
        try:
            job_ids.append(res.json()['id'])
        except: 
            pprint(res.json())

print(f'{len(job_ids)} new jobs posted')

57 are already downloaded.



1 new jobs posted


Below this cell write the code to download the completed files. First check if a job_id's status is completed and wait until it is. After it has been completed, download the file and save it.

The exact code required here is all in the documentation, the only additional task you have to do on your own is figure out a way to find out which file has just been received from the job_id, and name the local file.

**Please look at the Example JSON response in the [documentation](https://developers.zamzar.com/docs) to learn how to figure out the filenames, job status etc**

In [4]:
if not os.path.exists(csv_directory):
    os.mkdir(csv_directory)

In [12]:
## Your code goes here ##
for job_id in tqdm_notebook(job_ids, desc='Job ID'):
    endpoint = f"https://api.zamzar.com/v1/jobs/{job_id}"
    res = requests.get(endpoint, auth=HTTPBasicAuth(api_key, ''))
    
    while res.json()['status'] != 'successful': 
        # Wait for job to be done, since jobs were posted in order
        #   just waiting for the first job will be required. The rest 
        #   will probably be done by the time this loop exits
        pass
    
    target_file = res.json()['target_files'][0]
    file_id = target_file['id']
    local_filename = csv_directory + target_file['name']

    endpoint = f"https://api.zamzar.com/v1/files/{file_id}/content"
    res = requests.get(endpoint, stream=True, auth=auth_token)

    try:
        with open(local_filename, 'wb') as f:
            file_iter = res.iter_content(chunk_size=1024)
            for chunk in tqdm_notebook(file_iter, desc='Downloading', leave=False):
                if chunk:
                    f.write(chunk)
                    f.flush()
    except IOError as e:
        print(e)

## Part 3: Parsing the CSV File

We can not simply use Pandas read_csv here since the conversion is not perfect and there will be rows with different number of columns, which Pandas does not take care of.

**The following steps are performed:**
* Parsing CSV into pandas dataframe.
* Each DataFrame should consist of three columns with headers Bank, Donor_Name, and Amount
* The date should be retrieved from the given filename 
* The Donor_Name can be NaN, as it is in a lot of cases. 
* Remove all "Page of" rows
* Don't include the header rows (e.g. "SUPREME COURT FUND....") into the DataFrame
* The Amount should be converted into a Pandas numeric at the end
* Use regex to extract information. Also use it to correctly format the 3 columns.
* Use regex to extract the amount from the problem row and put the amount as the third column
* Merge the rest of the string as a name of the donor in the 2nd column

In [5]:
def parser_all(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        orig = [row for row in reader]
        
        # Convert list into string
        data = [' '.join(row) for row in orig[4:]]

        # Clean commas in the currency amounts
        data = [line.replace(',', '') for line in data]

        # Remove 'Total' and 'Page number' rows
        data = [row for row in data if 'Total' not in row and 'Page' not in row]

        records = []
        for line in data:
            amount = re.search(r' [1-9]\d*', line)
            amount = amount.group() if amount else ''

            bank = re.search(r'.*(LTD|Ltd|LIMITED|Limited|Bank|BANK|BSC|A/C)', line)
            bank = bank.group() if bank else ''
            
            account_num = re.search(r'[^\d]0\d*', line)
            account_num = account_num.group() if account_num else ''

            person = line.replace(amount, '').replace(bank, '').replace(account_num, '')

            records.append((bank.strip(), person.strip(), amount.strip(), line))

    return records

In [6]:
example_file = csv_directory + '17-08-2018.csv' # Assuming the file is in the folder all_csvs and is named appropriately
# This is one of the most problematic files which is why I have included this in the example

In [7]:
# Remember, remove headers and convert all amounts to Numeric; if it can't be converted it needs to be NaN
def read_csv(filename):
    headers = ['Bank', 'Donor_Name', 'Amount', 'Line']
    # ^ The entire line has been added to the DF to debug the cleaning process
    
    raw_data = parser_all(filename)
    
    df = pd.DataFrame.from_records(raw_data, columns=headers)
    df['Amount'] = pd.to_numeric(df['Amount'])
    
    return df
    
read_csv(example_file).head()

,Bank,Donor_Name,Amount,Line
0,AL BARAKA BANK (PAKISTAN) LTD,MOAZZAM ALI,200.0,AL BARAKA BANK (PAKISTAN) LTD MOAZZAM ALI 0117...
1,AL BARAKA BANK (PAKISTAN) LTD,MEHMOOD KHAN LODHI,500.0,AL BARAKA BANK (PAKISTAN) LTD MEHMOOD KHAN LOD...
2,AL BARAKA BANK (PAKISTAN) LTD,M UMAR,1000.0,AL BARAKA BANK (PAKISTAN) LTD M UMAR 0117 1000
3,AL BARAKA BANK (PAKISTAN) LTD,SAFDAR,1000.0,AL BARAKA BANK (PAKISTAN) LTD SAFDAR 0117 1000
4,AL BARAKA BANK (PAKISTAN) LTD,AMBREEN ZIA,1600.0,AL BARAKA BANK (PAKISTAN) LTD AMBREEN ZIA 0117...


## Part 4: Importing Full Dataset

* Run the parser on all the files
* For each file **add a 'Date' column, which should be inferred from the filename**
* Concatenate each DataFrame into one large DataFrame. 

In [8]:
files = glob.glob(csv_directory + '*.csv')

full_data = pd.DataFrame()
for file in tqdm_notebook(files):
    date = re.search(r'\d{2}-\d{2}-\d{4}', file)
    date = date.group()
    df = read_csv(file)  
    # Final Date format is %y %m %d
    df['Date'] = datetime.strptime(date, '%d-%m-%Y').date()
    df = df[['Bank', 'Donor_Name', 'Amount', 'Date', 'Line']]
    full_data = full_data.append(df)
full_data.reset_index(drop=True, inplace=True)

print(full_data.head())
print(full_data.shape)
print(full_data.tail())


                            Bank        Donor_Name  Amount        Date  \
0  AL BARAKA BANK (PAKISTAN) LTD       DAM UL HUDA  2000.0  2018-08-01   
1  AL BARAKA BANK (PAKISTAN) LTD       FARIS AHMED  2000.0  2018-08-01   
2  AL BARAKA BANK (PAKISTAN) LTD    MUHAMMAD AJMAL  1000.0  2018-08-01   
3  AL BARAKA BANK (PAKISTAN) LTD  SHAHEENA SULTANA  1000.0  2018-08-01   
4  AL BARAKA BANK (PAKISTAN) LTD       RASHID KHAN  1000.0  2018-08-01   

                                                Line  
0  AL BARAKA BANK (PAKISTAN) LTD DAM UL HUDA 2000.00  
1  AL BARAKA BANK (PAKISTAN) LTD FARIS AHMED 2000.00  
2  AL BARAKA BANK (PAKISTAN) LTD MUHAMMAD AJMAL 1...  
3  AL BARAKA BANK (PAKISTAN) LTD SHAHEENA SULTANA...  
4  AL BARAKA BANK (PAKISTAN) LTD RASHID KHAN 1000.00  
(174635, 5)
                            Bank       Donor_Name  Amount        Date  \
174630  Zarai Taraqiati Bank Ltd           M KHAN   500.0  2018-08-31   
174631  Zarai Taraqiati Bank Ltd  INAYAT ALI KHAN   500.0  2018-08

## Part 5: Data Integrity Checks

**Questions to ask?**
* How many NaN values are there in each column? Why are they there? 
* What are the maximum and minimum values, is there anything peculiar about the max values?
* Are there any rows which are not NaN but should still be a different DataFrame altogether?
* Should these problem rows be removed? Can they be useful in other ways?

In [56]:
# Using the following operation we can see that the rows where no Bank is present
# are the records in the data with missing Bank information. 
# People only wrote their name and address
full_data[full_data['Bank'] == '']

is_empty_string = lambda x: x == ''
empty_donor = np.where(full_data['Donor_Name'].apply(is_empty_string))
empty_bank = np.where(full_data['Bank'].apply(is_empty_string))
empty_amount = np.where(full_data['Amount'].apply(is_empty_string))

print('Empty Strings:')
print(f'Donor_name: {len(empty_donor[0])}\nBank: {len(empty_bank[0])}\nAmount: {len(empty_amount[0])}')

Empty Strings:
Donor_name: 3134
Bank: 6562
Amount: 0


In [52]:
# Using the following operation we can see that the rows where no Amount is present
# are the records in the data with missing currency information. 
full_data[full_data['Amount'].isna()]

# Includes Nan,None. Does not include empty strings
full_data.isna().sum()

Bank             0
Donor_Name       0
Amount        6449
Date             0
Line             0
dtype: int64

In [39]:
# Using the following operation we can see that the rows where no Amount is present
# are the records in the data with missing currency information. 
full_data[full_data['Amount'].isnull()]

full_data.isnull().sum()

Bank             0
Donor_Name       0
Amount        6449
Date             0
Line             0
dtype: int64

In [10]:
# Column-wise independant max
full_data.min(axis=0)

Bank                    
Donor_Name              
Amount                 1
Date          2018-07-06
Line                1000
dtype: object

In [12]:
# Column-wise independant max
full_data.max(axis=0)

Bank           salah ud din FIRST WOMEN BANK LTD
Donor_Name                                zumain
Amount                               3.74056e+17
Date                                  2018-10-04
Line          donations received from Muhamma   
dtype: object

In [13]:
# The row with min Amount
full_data.loc[full_data['Amount'].idxmin()]

Bank                                      Allied Bank Limited
Donor_Name                             H NO265 A F MIRPUR 200
Amount                                                      1
Date                                               2018-08-01
Line          Allied Bank Limited H NO265 A F 1 MIRPUR 200.00
Name: 140, dtype: object

In [14]:
# The row with max Amount
full_data.loc[full_data['Amount'].idxmax()]

Bank                                         Habib Bank Limited
Donor_Name           KASHIF ALI MAKHDOMI ARL MORGAH 03215384822
Amount                                              3.74056e+17
Date                                                 2018-09-26
Line          Habib Bank Limited KASHIF ALI MAKHDOMI ARL MOR...
Name: 148170, dtype: object

The rows containing the total amount for the corresponding banks should either be kept in a different dataframe or calculated using a grouping operation. We have already removed these rows from our dataset.

Another set of problematic rows are the donations made using SMS. Since they do not have a bank associated with them directly, the `Bank` column will be empty for these rows. It would be better to keep these donations in a separate dataframe. 